! jt -r

In [159]:
from importlib import reload
reload(aux)
reload(con)
reload(dt)

<module 'src.daily_data' from 'C:\\Users\\ugur.eren\\Python Codes\\cefis2\\src\\daily_data.py'>

In [160]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io
import src.density as den

1. **data** klasöründeki tüm dosya yolları alınıyor

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [3]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
> Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [4]:
paired_days = dt.to_match_days(infolist)

4.  **gunluk_veriler**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor

In [5]:
gunluk_veriler = []

In [6]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.GunlukVeri(saatik_data=paired_hours,tarih=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    gunluk_veriler.append(gunluk_veri)

5. Her bir **GunlukVeri** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
   > (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [161]:
products={}

In [162]:
for gunluk in gunluk_veriler:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:        
        products[gunluk.name] = [gunluk]

In [163]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [164]:
stop=False
while not stop:
    try:
        urun = products.popitem()
    except:
        stop=True
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
    combined_days_detail[urun[0]] = aux.get_detail(df)
    combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

In [184]:
all_products = list(combined_days.keys())

In [195]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

In [198]:
cd_density= {}

In [200]:
for product in all_products:
    densities = []
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        density = den.calc_density(df)
        densities.append(density)
    cd_density[product] = pd.concat(densities,keys=hour_series)